# Improved bounds

We can improve the bounds by trying to incorporate the negative weights.

We do this by pairing them with their nearest positive weight, and replacing the two of them with another weight, placed at the maximum of their sum, with the weight equal to the height of this peak.

Is this valid?

Consider a positive RBF at $0$, with weight $A$, and a negative RBF at $b$ with weight $-B$. We replace these with a new RBF with weight $C$ at $c$.

For positive $x$,...

TO DO We probably need to prove this aspect.

In [2]:

import numpy as np
from boundmixofgaussians import zeromean_gaussian, findbound, PCA, findbound_lowdim

def test():
    ls = 2.0
    W = 1.0*np.array([3,3])
    approx = findbound(X=1.0*np.array([[0,0,0,0],[1,2,1,1]]),W=W,ls=ls,d=4,gridres=30,gridstart=1.0*np.array([0,0,0,0]),gridend=1.0*np.array([1,1,1,1]),fulldim=True)
    correct = 3*2*np.exp(-.5*(1**2+3*0.5**2)/ls**2)
    assert np.abs(approx-correct)<0.001

    #as there's just two points a 1d manifold will be correct too.
    approx = findbound(X=1.0*np.array([[0,0,0,0],[1,2,1,1]]),W=W,ls=ls,d=4,gridres=10000,gridstart=1.0*np.array([0,0,0,0]),gridend=1.0*np.array([1,1,1,1]),dimthreshold=1)
    assert np.abs(approx-correct)<1e-6


    W = 1.0*np.array([1]*7)
    ls = 0.3
    #the biggest value in 2d is a single peak from a single training point
    approx2d = findbound(X=1.0*np.array([[0,0],[1+.5,1-.5],[1-.5,1+.5],[2,2],[3,3],[4,4]]),W=np.array([1,1,0.5,1,1,1]),ls=ls,d=2,gridres=300,gridstart=1.0*np.array([0,0]),gridend=1.0*np.array([4,4]),dimthreshold=2)
    assert np.abs(approx2d-1)<0.01

    #when dimensionality is reduced to 1d two peaks cancel out.
    approx1d = findbound(X=1.0*np.array([[0,0],[1+.5,1-.5],[1-.5,1+.5],[2,2],[3,3],[4,4]]),W=np.array([1,1,0.5,1,1,1]),ls=ls,d=2,gridres=300,gridstart=1.0*np.array([0,0]),gridend=1.0*np.array([4,4]),dimthreshold=1)
    assert np.abs(approx1d-1.5)<0.01

    #the negative peak shouldn't cancel out the positive one.
    approx1d = findbound(X=1.0*np.array([[0,0],[1+.5,1-.5],[1-.5,1+.5],[2,2],[3,3],[4,4]]),W=np.array([1,2,-1,1,1,1]),ls=ls,d=2,gridres=300,gridstart=1.0*np.array([0,0]),gridend=1.0*np.array([4,4]),dimthreshold=1)
    #assert np.abs(approx1d-2.0)<0.01
    #it should now!
    assert np.abs(approx1d-1.0)<0.01

    #the negative peak shouldn't cancel out the positive one if they're just really close (as that's not been coded)
    approx1d = findbound(X=1.0*np.array([[0,0],[1,1],[2,2],[3,3.01],[3,3]]),W=np.array([1,1,1,2,-1]),ls=ls,d=2,gridres=300,gridstart=1.0*np.array([0,0]),gridend=1.0*np.array([4,4]),dimthreshold=1)
    approx1daligned = findbound(X=1.0*np.array([[0,0],[1,1],[2,2],[3,3],[3,3]]),W=np.array([1,1,1,2,-1]),ls=ls,d=2,gridres=300,gridstart=1.0*np.array([0,0]),gridend=1.0*np.array([4,4]),dimthreshold=1)
    approx2d = findbound(X=1.0*np.array([[0,0],[1,1],[2,2],[3,3.01],[3,3]]),W=np.array([1,1,1,2,-1]),ls=ls,d=2,gridres=300,gridstart=1.0*np.array([0,0]),gridend=1.0*np.array([4,4]),dimthreshold=2)
    assert np.abs(approx2d-1.0)<0.01 #negative cancels out +ve peak (2-1) in 2d
    
    assert np.abs(approx1daligned-1.0)<0.01 #negative will cancel out +ve peak if correctly aligned (in 1d)
    #assert np.abs(approx1d-2.0)<0.01 #negative ignored in 1d
    
    assert np.abs(approx1d-1.0)<0.01 #negative cancels!
test()

In [193]:
np.set_printoptions(suppress=True,precision=5)
from boundmixofgaussians import zeromean_gaussian_1d

def compute_sum(w1,w2,x,ls,dist):
    return zeromean_gaussian_1d(x,ls,w1)+zeromean_gaussian_1d(x-dist,ls,w2)
def compute_grad(w1,w2,x,ls,dist):
    return (((x/ls**2)*zeromean_gaussian_1d(x,ls,w1))+(((x-dist)/ls**2)*zeromean_gaussian_1d(x-dist,ls,w2)))


def findpeak(w1,w2,dist,ls):
    #print(w1,w2,dist,ls)
    assert w1>0
    assert w2<0
    x = 0
    lr = 0.2
    #for it in range(20):
    oldx = np.inf
    while np.abs(oldx-x)>1e-10:
       # print("%d (%05f %05f)" % (x,(compute_sum(w1,w2,x,ls,dist)-compute_sum(w1,w2,x+0.0000001,ls,dist))/0.0000001,compute_grad(w1,w2,x,ls,dist)))
        oldx = x
        x-=lr*compute_grad(w1,w2,x,ls,dist)
        lr*=0.99999
        
    return x, compute_sum(w1,w2,x,ls,dist)

def mergenegatives(X,W,ls):
    
    i = -1
    #for i,(x,w) in enumerate(zip(X,W)): #find all negatives
    while(i<len(X)-1):
        i+=1
        x = X[i,:]
        w = W[i]   
        
        if w>=0:
            continue
        sqrdists = np.sum((x-X)**2,1)
        sqrdists[W<=0] = np.infty #we want to combine with a positive weight
        nearestpositive = np.argmin(sqrdists)
        dist = np.sqrt(sqrdists[nearestpositive])
        offset,newpeak = findpeak(W[nearestpositive],w,dist,ls)
        print(offset,newpeak)
        vector = (x - X[nearestpositive,:])
        #offset value is positive in direction from +ve weight to -ve weight
        newx = (X[nearestpositive,:] + offset * vector/dist)
        #delete nearestpositive and i from X and W
        #add newx and newpeak to X and W
        #do so by replacing one, deleting other
        #
        #so in effect, move +ve one:
        X[nearestpositive,:] = newx
        W[nearestpositive] = newpeak
        #delete negative one:
        #X = np.delete(X,i,0)
        #W = np.delete(W,i,0)
        #for speed we'll just set W to zero, and delete later...
        W[i] = 0
    X = np.delete(X,np.where(W==0),0)
    W = np.delete(W,np.where(W==0),0)
    return (X,W)

In [4]:
from boundmixofgaussians import zeromean_gaussian_1d, compute_grad, compute_sum, mergenegatives

assert np.abs(((compute_sum(3,-1,0.1,1.2,1.2)-compute_sum(3,-1,0.1+0.0000001,1.2,1.2))/0.0000001)-compute_grad(3,-1,0.1,1.2,1.2))<0.00001
#if ls is large relative to distances, then the solution
#to the gradient=0 is at A.x = B.x-B.dist (A-B)x = dist*B/(A-B)
#in the case below dist = 0.02236068

dist = 0.02236068
X=1.0*np.array([[0,0],[1,1],[2,2],[3.02,3.01],[3,3]])
W=1.0*np.array([1,1,1,4,-2])
#here A=4, B=2: 2/(4-2)=2/2=1
newX,newW = mergenegatives(X.copy(),W,10.0)
assert (newX[3,1]-(dist*1+X[3,1])<1e-5)


X=1.0*np.array([[0,0],[1,1],[2,2],[3,3.01],[3,3]])
W=1.0*np.array([1,1,1,5,-1])
#here A=5, B=1: 1/(5-1)=1/4
newX,newW = mergenegatives(X.copy(),W,5.0)
assert (newX[3,1]-(dist*(1/4)+X[3,1])<1e-5)
#note that the lengthscale doesn't come into the calc.

#Test that a distant negative will have no impact
X=1.0*np.array([[0,0],[1,1],[2,2],[3,103],[3,3]])
W=1.0*np.array([1,1,1,-2,4])
newX,newW = mergenegatives(X.copy(),W,2.0)
assert (np.all(newW==np.array([1,1,1,4])))